# Supabase (Postgres)

>[Supabase](https://supabase.com/docs) 是一个开源的 `Firebase` 替代品。
> `Supabase` 构建在 `PostgreSQL` 之上，它提供了强大的 `SQL` 查询能力，并支持与现有工具和框架的简单集成。

>[PostgreSQL](https://en.wikipedia.org/wiki/PostgreSQL)，也称为 `Postgres`，
>是一个免费且开源的关系数据库管理系统（RDBMS），其特点是可扩展性和 `SQL` 合规性。
>
>[Supabase](https://supabase.com/docs/guides/ai) 提供了一个开源工具包，用于使用 Postgres 和 pgvector 开发 AI 应用程序。
>使用 Supabase 客户端库可以大规模地存储、索引和查询您的向量嵌入。

在本笔记本中，我们将演示围绕 `Supabase` 向量存储包装的 `SelfQueryRetriever`。

具体来说，我们将：
1. 创建一个 Supabase 数据库
2. 启用 `pgvector` 扩展
3. 创建一个 `documents` 表和 `match_documents` 函数，供 `SupabaseVectorStore` 使用
4. 将示例文档加载到向量存储（数据库表）中
5. 构建并测试一个自查询检索器

## 设置 Supabase 数据库

1. 前往 https://database.new 预配你的 Supabase 数据库。
2. 在 Studio 中，跳转到 [SQL 编辑器](https://supabase.com/dashboard/project/_/sql/new) 并运行以下脚本来启用 `pgvector` 并将你的数据库设置为向量存储：
    ```sql
    -- Enable the pgvector extension to work with embedding vectors
    create extension if not exists vector;

    -- Create a table to store your documents
    create table
      documents (
        id uuid primary key,
        content text, -- corresponds to Document.pageContent
        metadata jsonb, -- corresponds to Document.metadata
        embedding vector (1536) -- 1536 works for OpenAI embeddings, change if needed
      );

    -- Create a function to search for documents
    create function match_documents (
      query_embedding vector (1536),
      filter jsonb default '{}'
    ) returns table (
      id uuid,
      content text,
      metadata jsonb,
      similarity float
    ) language plpgsql as $$
    #variable_conflict use_column
    begin
      return query
      select
        id,
        content,
        metadata,
        1 - (documents.embedding <=> query_embedding) as similarity
      from documents
      where metadata @> filter
      order by documents.embedding <=> query_embedding;
    end;
    $$;
    ```

## 创建 Supabase 向量存储
接下来，我们将创建 Supabase 向量存储并为其填充数据。我们创建了一组包含电影摘要的小型演示文档。

请确保安装最新版本的 `langchain` 并支持 `openai`：

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai tiktoken

self-query retriever 要求您安装 `lark`：

In [ ]:
%pip install --upgrade --quiet  lark

我们还需要 `supabase` 包：

In [ ]:
%pip install --upgrade --quiet  supabase

由于我们使用了 `SupabaseVectorStore` 和 `OpenAIEmbeddings`，因此需要加载它们的 API 密钥。

- 要查找你的 `SUPABASE_URL` 和 `SUPABASE_SERVICE_KEY`，请前往你的 Supabase 项目的 [API 设置](https://supabase.com/dashboard/project/_/settings/api)。
  - `SUPABASE_URL` 对应于 Project URL
  - `SUPABASE_SERVICE_KEY` 对应于 `service_role` API 密钥

- 要获取你的 `OPENAI_API_KEY`，请前往 OpenAI 账户中的 [API keys](https://platform.openai.com/account/api-keys) 并创建一个新的密钥。

In [1]:
import getpass
import os

if "SUPABASE_URL" not in os.environ:
    os.environ["SUPABASE_URL"] = getpass.getpass("Supabase URL:")
if "SUPABASE_SERVICE_KEY" not in os.environ:
    os.environ["SUPABASE_SERVICE_KEY"] = getpass.getpass("Supabase Service Key:")
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

_可选：_ 如果您将 Supabase 和 OpenAI API 密钥存储在 `.env` 文件中，您可以使用 [`dotenv`](https://github.com/theskumar/python-dotenv) 来加载它们。

In [ ]:
%pip install --upgrade --quiet  python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv()

首先，我们将创建一个 Supabase 客户端，并实例化一个 OpenAI embeddings 类。

In [2]:
import os

from langchain_community.vectorstores import SupabaseVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

接下来，我们来创建文档。

In [3]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]

vectorstore = SupabaseVectorStore.from_documents(
    docs,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)

## 创建我们的自查询检索器
现在我们可以实例化我们的检索器了。为此，我们需要提前提供有关我们的文档支持的元数据字段以及文档内容的简短描述的一些信息。

In [4]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## 测试一下
现在我们可以尝试实际使用我们的检索器了！<ctrl63>

In [5]:
# This example only specifies a relevant query
retriever.invoke("What are some movies about dinosaurs")

query='dinosaur' filter=None limit=None


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'})]

In [7]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

query=' ' filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=8.5) limit=None


[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'})]

In [9]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women?")

query='women' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Greta Gerwig') limit=None


[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'rating': 8.3, 'director': 'Greta Gerwig'})]

In [8]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='rating', value=8.5), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction')]) limit=None


[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'})]

In [9]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before (or on) 2005 that's all about toys, and preferably is animated"
)

query='toys' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='year', value=2005), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='genre', value='animated')]) limit=None


[Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'})]

## Filter k

我们还可以使用 self query retriever 来指定 `k`：要获取的文档数量。

我们可以通过向构造函数传递 `enable_limit=True` 来实现此目的。

In [10]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [11]:
# This example only specifies a relevant query
retriever.invoke("what are two movies about dinosaurs")

query='dinosaur' filter=None limit=2


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'})]